In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512,'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}
# viso
# draw.io

In [3]:
def make_feature(cfg):
    feature_layers = []
    for v in cfg:
        if v == 'M':
            feature_layers.append(keras.layers.MaxPool2D())
        else:
            feature_layers.append(keras.layers.Conv2D(v, kernel_size=3, padding='SAME', activation='relu'))
            
    return keras.Sequential(feature_layers, name='feature')
                                  

In [10]:
def VGG(feature, im_height=224, im_width=224, num_classes=1000):
    input_image = keras.layers.Input(shape=(im_height, im_width, 3), dtype='float32')
    x = feature(input_image)
    x = keras.layers.Flatten()(x)
    
    x = keras.layers.Dense(2048, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x)
    x = keras.layers.Dense(2048, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x)
    x = keras.layers.Dense(num_classes)(x)
    output = keras.layers.Softmax()(x)
    
    model = keras.models.Model(inputs=input_image, outputs=output)
    return model

In [11]:
def vgg(model_name='vgg16', im_height=224, im_width=224, num_classes=1000):
    cfg = cfgs[model_name]
    model = VGG(make_feature(cfg), im_height=im_height, im_width=im_width, num_classes=num_classes)
    return model

In [6]:
train_dir = r'D:\AIoT-深度学习视频版\深度学习二期\day12_ALexNet和VGG\代码\training\training'
valid_dir = r'D:\AIoT-深度学习视频版\深度学习二期\day12_ALexNet和VGG\代码\validation\validation'
label_file = r'D:\AIoT-深度学习视频版\深度学习二期\day12_ALexNet和VGG\代码\monkey_labels.txt'

In [7]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0 / 255.0,
    # 以下都是图片数据增强操作.
#     rotation_range = 40,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     shear_range = 0.2,
#     zoom_range = 0.2,
    horizontal_flip = True, 
    vertical_flip = True,
    
    fill_mode = 'nearest'
)

# 从目录中读取图片
height = 224
width = 224
channels = 3
batch_size = 64
num_classes = 10

# 会自动把目录名作为label名. 
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(height, width),
                                 batch_size=batch_size,
                                 shuffle=True,
                                 seed=7,
                                 class_mode='categorical')

Found 1098 images belonging to 10 classes.


In [16]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(height, width),
                                 batch_size=batch_size,
                                 shuffle=True,
                                 seed=7,
                                 class_mode='categorical')

Found 272 images belonging to 10 classes.


In [17]:
vgg11 = vgg(model_name='vgg11', num_classes=10)

In [18]:
vgg11.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 feature (Sequential)        (None, 7, 7, 512)         9220480   
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 2048)              51382272  
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0   

In [19]:
vgg11.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [20]:
train_num = train_generator.samples
valid_num = valid_generator.samples

history = vgg11.fit(train_generator,
                   steps_per_epoch= train_num // batch_size,
                   epochs=30,
                   validation_data=valid_generator,
                   validation_steps = valid_num // batch_size)

Epoch 1/30
17/17 [==============================] - 17s 946ms/step - loss: 2.4729 - acc: 0.1083 - val_loss: 2.3033 - val_acc: 0.1133
Epoch 2/30
17/17 [==============================] - 15s 882ms/step - loss: 2.3035 - acc: 0.1103 - val_loss: 2.3023 - val_acc: 0.1055
Epoch 3/30
17/17 [==============================] - 15s 898ms/step - loss: 2.3029 - acc: 0.1161 - val_loss: 2.3030 - val_acc: 0.1055
Epoch 4/30
17/17 [==============================] - 16s 918ms/step - loss: 2.3019 - acc: 0.1180 - val_loss: 2.3028 - val_acc: 0.1055
Epoch 5/30
17/17 [==============================] - 15s 882ms/step - loss: 2.3029 - acc: 0.1083 - val_loss: 2.3019 - val_acc: 0.1094
Epoch 6/30
 3/17 [====>.........................] - ETA: 9s - loss: 2.2995 - acc: 0.1354

KeyboardInterrupt: 